In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
from tqdm import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
model = models.vgg16_bn(pretrained=True)

In [3]:
from kaishi.core.image.nn import Model
M = Model()
model = M.model

FileNotFoundError: [Errno 2] No such file or directory: '/home/mwharton/Code/kaishi/kaishi/image_macro_issues_vgg16.pth'

In [ ]:
#for param in model.parameters():
#    param.requires_grad = False

In [ ]:
n_classes = 6
n_inputs = model.classifier[-1].in_features

In [ ]:
model.classifier[6] = nn.Sequential(nn.Linear(n_inputs, 256),
                                    nn.ReLU(),
                                    nn.Dropout(0.15),
                                    nn.Linear(256, n_classes),
                                    nn.Sigmoid())

In [ ]:
from kaishi.image import Dataset
imdata = Dataset('/home/mwharton/Data/kaishi_image_train_dataset')
imdata.run_pipeline()

In [ ]:
model = model.to('cuda')
steps_per_epoch = 40
data_generator = imdata.train_generator(batch_size=16)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1e-4)
n_epochs = 30

In [ ]:
X_val, y_val = imdata.generate_validation_data()

In [ ]:
minval = 9999

In [ ]:
for epoch in range(n_epochs):
    print('Epoch %3d/%3d' % (epoch + 1, n_epochs))
    running_loss = 0
    lossct = 0
    for step in tqdm(range(steps_per_epoch)):
        optimizer.zero_grad()
        for X, y in data_generator:
            break
        pred = model(torch.from_numpy(X).to(torch.float32).to('cuda'))
        loss = criterion(pred, torch.from_numpy(y).to(torch.float32).to('cuda'))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        lossct += 1
    print('Loss for this epoch: ' + repr(running_loss / float(lossct)))
    val_loss = 0
    for i in range(10):
        Xtmp = X_val[i*32:(i+1)*32]
        ytmp = y_val[i*32:(i+1)*32]
        val_pred = model(torch.from_numpy(Xtmp).to(torch.float32).to('cuda'))
        val_loss += criterion(val_pred, torch.from_numpy(ytmp).to(torch.float32).to('cuda')).item()
    print('Validation loss: ' + repr(val_loss / 10.0))
    if val_loss < minval:
        minval = val_loss
        print('Improved.')
        torch.save(model.state_dict(), 'best_val.pth')

In [ ]:
model.load_state_dict(torch.load('best_val.pth'))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
pred = model(torch.from_numpy(X_val[:10]).to(torch.float32).to('cuda')).cpu().detach().numpy()

In [ ]:
for i in range(32):
    plt.imshow(np.swapaxes(X_val[i], 0, 2))
    plt.show()
    print('Truth: ' + repr(y_val[i]))
    print('Pred:  ' + repr(pred[i]))

In [ ]:
# [DOCUMENT, RECTIFIED, ROTATED_RIGHT, ROTATED_LEFT, UPSIDE_DOWN, STRETCHING]